In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import random

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [7]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [8]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [9]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [10]:
len(allRatings)

200000

In [11]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

# Read prediction                                #

In [10]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [11]:
totalRead

200000

In [12]:
len(return1)

512

In [13]:
len(mostPopular)

6688

In [14]:
bookPeruser=defaultdict(set)
allbook=set()
for u,b,r in ratingsTrain:
    bookPeruser[u].add(b)
    allbook.add(b)
allbook=list(allbook)
i=0
n=len(ratingsValid)
while i<n:
    #print(n)
    u,b,_=ratingsValid[i]
    nb=random.choice(allbook)
    while nb in bookPeruser[u]:
        nb=random.choice(allbook)
    ratingsValid.append((u,nb,None))
    i+=1
len(ratingsValid)

20000

In [15]:
ratingsValid[-1]

('u55163096', 'b13528908', None)

In [18]:
def get_acc(predictions,ratingsValid):
    n=len(predictions)
    pos=0
    for i in range(n):
        if (predictions[i]==True and ratingsValid[i][2]!=None) or (predictions[i]==False and ratingsValid[i][2]==None):
            pos+=1
    return pos/n

In [19]:
def model_on_different_threshold(thre):
    bookCount = defaultdict(int)
    totalRead = 0

    for user,book,_ in readCSV("train_Interactions.csv.gz"):
        bookCount[book] += 1
        totalRead += 1

    mostPopular = [(bookCount[x], x) for x in bookCount]
    mostPopular.sort()
    mostPopular.reverse()

    returnthre = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        returnthre.add(i)
        if count > totalRead*thre: break
    predictions=[]
    for u,b,_ in ratingsValid:
        if b in returnthre:
            predictions.append(True)
        else:
            predictions.append(False)
    return predictions

In [20]:
acclist=[]
for i in range(1,200):
    thre=0.005*i
    predictions=model_on_different_threshold(thre)
    acc=get_acc(predictions,ratingsValid)
    acclist.append(acc)
max(acclist)

0.7525

In [21]:
acclist.index(max(acclist))

147

In [22]:
threshold=0.005*(acclist.index(max(acclist))+1)
acc2=max(acclist)

In [23]:
usersPerbook = defaultdict(set) 
booksPeruser = defaultdict(set) 

for u,b,_ in ratingsTrain:
    usersPerbook[b].add(u)
    booksPeruser[u].add(b)

In [24]:
def Jaccard(s1, s2):
    nu=len(s1.intersection(s2))
    de=len(s1.union(s2))
    return nu/de

In [26]:
def predict(u,b,threshold):
    similarities=[]
    for ob in booksPeruser[u]:
        if ob==b:continue
        similarities.append(Jaccard(usersPerbook[ob],usersPerbook[b]))
    if max(similarities,default=0)>threshold:
        return True
    else:
        return False

In [27]:
def model(threshold):
    predictions=0
    for u,b,r in ratingsValid:
        if predict(u,b,threshold)==True and r!=None:
            predictions+=1
        elif predict(u,b,threshold)==False and r==None:
            predictions+=1
        else:
            pass
    return predictions/len(ratingsValid)

In [32]:
def model_on_different_threshold2(thre1,thre2):
    bookCount = defaultdict(int)
    totalRead = 0

    for user,book,_ in readCSV("train_Interactions.csv.gz"):
        bookCount[book] += 1
        totalRead += 1

    mostPopular = [(bookCount[x], x) for x in bookCount]
    mostPopular.sort()
    mostPopular.reverse()

    returnthre = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        returnthre.add(i)
        if count > totalRead*thre1: break
    predictions=[]
    for u,b,_ in ratingsValid:
        if b in returnthre or predict(u,b,thre2)==True:
            predictions.append(True)
        else:
            predictions.append(False)
    return predictions

#select the best pair of thresholds
#this procedure takes at least ten minutes since pairs are too many
acclist2=[]
thresholds=[]
for i in range(100,200):
    for j in range(1,9):
        thre1=i*0.005
        thre2=j*0.01
        predictions=model_on_different_threshold2(thre1,thre2)
        acc=get_acc(predictions,ratingsValid)
        thresholds.append((thre1,thre2))
        acclist2.append(acc)
        #print(acc)
max(acclist2)

0.7533

In [33]:
acc=max(acclist2)
acc

0.7533

In [34]:
index=acclist2.index(acc)
index

388

In [35]:
thre1,thre2=thresholds[index]
(thre1,thre2)

(0.74, 0.05)

In [36]:
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

returnthre = set()
count = 0
for ic, i in mostPopular:
    count += ic
    returnthre.add(i)
    if count > totalRead*thre1: break

In [37]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    
    if b in returnthre or predict(u,b,thre2)==True:
        predictions.write(u + ',' + b + ",1\n")
    else:
        predictions.write(u + ',' + b + ",0\n")
    '''
    
    if b in return1:
        predictions.write(u + ',' + b + ",1\n")
    else:
        predictions.write(u + ',' + b + ",0\n")
    '''
predictions.close()

# Rating Prediction

In [175]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [176]:
ratingMean=0
for u,b,r in ratingsTrain:
    ratingMean+=r
ratingMean/=len(ratingsTrain)
ratingMean

3.6868052631578947

In [177]:
alpha = ratingMean

In [178]:
N = len(ratingsTrain)
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())

In [179]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [180]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(ratingsTrain)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for u,b,r in ratingsTrain:
        
        pred = prediction(u, b)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[b] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[b] += 2*lamb*itemBiases[b]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[b] for b in items]
    return numpy.array(dtheta)

In [181]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u, b) for u,b,_ in ratingsTrain]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [182]:
labels = [r for _,_,r in ratingsTrain]

In [183]:
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error

In [184]:
def prediction(user, item):
    if user in userBiases and item in itemBiases:
        return alpha + userBiases[user] + itemBiases[item]
    elif item in itemBiases:
        return alpha + itemBiases[item]
    elif user in userBiases:
        return alpha + userBiases[user]
    else:
        return alpha

In [185]:
validlabels = [r for _,_,r in ratingsValid]

In [191]:
MSES=[float('inf')]
bestlamb=float('inf')
for i in range(10):
    lamb=1e-4*i
    scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),derivative, args = (labels, lamb))
    predictions=[]
    for u,b,r in ratingsValid:
        predict=prediction(u, b)
        predictions.append(predict)
    validMSE=MSE(predictions,validlabels)
    
    MSES.append((validMSE,lamb))

MSE = 1.7734685938815473
MSE = 2.4112002848567267
MSE = 1.7412249889894722
MSE = 1.7409232480676344
MSE = 1.7397224767691113
MSE = 1.7350184697962532
MSE = 1.7177876934445695
MSE = 1.62036936752244
MSE = 1.5667237947029857
MSE = 1.4527543089531503
MSE = 1.3730004233764552
MSE = 1.3131598206774697
MSE = 1.26077478558939
MSE = 1.2277332811934907
MSE = 1.2090353324687886
MSE = 1.154553750304782
MSE = 1.144673457396151
MSE = 1.1274878865285929
MSE = 7.766303161382303
MSE = 1.1274155892533249
MSE = 1.1192943196346097
MSE = 1.108922431486299
MSE = 1.0998293373357884
MSE = 1.087974013630877
MSE = 1.0910181258117122
MSE = 1.0826035052484217
MSE = 1.0753859043927827
MSE = 1.070766324405037
MSE = 1.0660827831770354
MSE = 1.0625034654219547
MSE = 1.0597729342409044
MSE = 1.0570767028829735
MSE = 1.0619600300333607
MSE = 1.0563216789248735
MSE = 1.0558380023586267
MSE = 1.0534480168628366
MSE = 1.0520029754802436
MSE = 1.0506642074388053
MSE = 1.0491973834981112
MSE = 1.0482054047213565
MSE = 1.04

In [192]:
bestlamb

inf

In [196]:

MSES

[inf,
 (1.5393782297845815, 0.0),
 (1.4476733176191543, 0.0001),
 (1.4961960104138818, 0.0002),
 (1.5197645999868412, 0.00030000000000000003),
 (1.5402186477093065, 0.0004),
 (1.5516977476901797, 0.0005),
 (1.5598744486414595, 0.0006000000000000001),
 (1.5744061928053703, 0.0007),
 (1.5784065876324527, 0.0008),
 (1.588698185938979, 0.0009000000000000001)]

In [197]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),derivative, args = (labels, 1e-4))

MSE = 1.7481471086855647
MSE = 2.5623457319917486
MSE = 1.7412495457782147
MSE = 1.740942248189603
MSE = 1.7397194798988762
MSE = 1.7349311597554913
MSE = 1.7174219274943112
MSE = 1.6262610338465973
MSE = 1.708396903501915
MSE = 1.6634237474950306
MSE = 1.6438982472945143
MSE = 1.568436626243956
MSE = 1.5400141111531067
MSE = 1.4453137752644314
MSE = 1.3829298892837514
MSE = 1.3512220245192619
MSE = 1.352461511843311
MSE = 1.347197859156403
MSE = 2.4860407078203166
MSE = 1.3471212406776567
MSE = 1.3386328457262469
MSE = 1.3452807690205715
MSE = 1.3302021660082497
MSE = 1.3429902354863104
MSE = 1.3448820107406898
MSE = 1.3452095853806785
MSE = 1.3452680045561807
MSE = 1.3452784782837994
MSE = 1.3452803578607968
MSE = 1.345280695220406
MSE = 1.3452807557739261
MSE = 1.34528076664288
MSE = 1.345280768593802
MSE = 1.3452807689439705
MSE = 1.3452807690068178
MSE = 1.3452807690180963
MSE = 1.345280769020133
MSE = 1.3452807690204924
MSE = 1.345280769020255
MSE = 1.34528076902045
MSE = 1.34528

(array([ 3.53710495, -1.40611373,  0.09108505, ...,  0.08072248,
         0.02445632,  0.02548934]),
 1.4938572715500822,
 {'grad': array([-6.76786396e-04, -3.59621076e-05, -1.83186483e-06, ...,
         -1.25264680e-05, -4.15127749e-06, -4.42350434e-06]),
  'task': 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 145,
  'nit': 34,
  'warnflag': 0})

In [198]:
predictions=[]
for u,b,r in ratingsValid:
    predict=prediction(u, b)
    predictions.append(predict)
validMSE=MSE(predictions,validlabels)
validMSE

1.4509680044491837

In [ ]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    
    predictions.write(u + ',' + b + ',' + str(prediction(u,b)) + '\n')
    
predictions.close()

Try latent factor model with gamma instead of simple alpha

In [49]:
alpha = ratingMean
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [50]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [118]:
K=4
userGamma = {}
itemGamma = {}
for u,b,r in ratingsTrain:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]
    itemGamma[b] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [119]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

In [120]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [141]:
def prediction(user, item):
    if user in userBiases and item in itemBiases:
        return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])
    elif item in itemBiases:
        return alpha + itemBiases[item]+sum(itemGamma[item])
    elif user in userBiases:
        return alpha + userBiases[user]+sum(userGamma[user])
    else:
        return alpha

In [122]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u, b) for u,b,_ in ratingsTrain]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

In [123]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(ratingsTrain)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in ratingsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in ratingsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for u,b,r in ratingsTrain:
        pred = prediction(u, i)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta)

In [57]:
def training_with_l(l):
    scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(nUsers+nItems) + # Initialize beta
                                   [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
                             derivative, args = (labels, l))

In [66]:
bestmse=float('inf')
bestl=0
for i in range(20):
    l=i*1e-5
    predictions=[]
    training_with_l(l)
    for u,b,r in ratingsValid:
        predict=prediction(u, b)
        predictions.append(predict)
    validMSE=MSE(predictions,validlabels)
    if validMSE<bestmse:
        bestmse=validMSE
        bestl=l

MSE = 1.7418188605535554
MSE = 2.1924399255461493
MSE = 1.7778987863950753
MSE = 1.7446310652173855
MSE = 1.7417624474061344
MSE = 1.7416038028844596
MSE = 1.741576165189197
MSE = 1.7414656244368523
MSE = 1.7410236218867468
MSE = 1.7392581790347084
MSE = 1.7322374852007367
MSE = 1.7048119510516642
MSE = 1.6056256734444716
MSE = 1.289588108825058
MSE = 1.5484468931201352
MSE = 1.3647194355163998
MSE = 1.3134957284601216
MSE = 1.2957988456449665
MSE = 1.2906502482071658
MSE = 1.2897825271925436
MSE = 1.2896254032322174
MSE = 1.2895953424257138
MSE = 1.2895895149852732
MSE = 1.2895883822920975
MSE = 1.2895881620128742
MSE = 1.2895881191699634
MSE = 1.2895881108371243
MSE = 1.2895881092164017
MSE = 1.2895881089011738
MSE = 1.2895881088398624
MSE = 1.2895881088279375
MSE = 1.2895881088256178
MSE = 1.2895881088251668
MSE = 1.2895881088250791
MSE = 1.2895200229872577
MSE = 1.28926257289945
MSE = 1.288471064762527
MSE = 1.2976249535757012
MSE = 1.2880689135090206
MSE = 1.2848530231720163
MSE =

In [74]:
bestmse

1.471655135665015

In [69]:
bestl

3.0000000000000004e-05

In [124]:
training_with_l(bestl)

MSE = 1.7416711163334977
MSE = 2.1639997874381454
MSE = 1.7750369301219469
MSE = 1.7442962695818724
MSE = 1.7417414789844856
MSE = 1.7415235940293887
MSE = 1.741564908553344
MSE = 1.7415350289261327
MSE = 1.7414155221413117
MSE = 1.7409376825863758
MSE = 1.739029325716208
MSE = 1.7314439198287452
MSE = 1.7018706417703
MSE = 1.5958710574029804
MSE = 1.3280176164194386
MSE = 1.2497490313525628
MSE = 1.2777164170823736
MSE = 1.2981310628520186
MSE = 1.3128272310931777
MSE = 1.2840775287946138
MSE = 1.2957677199971862
MSE = 1.3029513159862232
MSE = 1.307454275623314
MSE = 1.3109559989741164
MSE = 1.312390603801793
MSE = 1.3114420835719611
MSE = 1.312179840827691
MSE = 1.3116924773658385
MSE = 1.3120742109449264
MSE = 1.3121573981664039
MSE = 1.3121024889842325
MSE = 1.3121573981664039
MSE = 1.3251889381594872
MSE = 1.3156454315450694
MSE = 1.359797438588689
MSE = 1.3274079412407178
MSE = 1.5887798344035542
MSE = 1.395175709986625
MSE = 1.3366651465945774
MSE = 1.3291927085064397
MSE = 1.32

In [125]:
predictions=[]
for u,b,r in ratingsValid:
        predict=prediction(u, b)
        predictions.append(predict)
validMSE=MSE(predictions,validlabels)
validMSE

1.4751249358054166

In [77]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    
    predictions.write(u + ',' + b + ',' + str(prediction(u,b)) + '\n')
    
predictions.close()

In [ ]:
bestl=0
bestmse=float('inf')
bestK=0
def train_model(K,l):
    userGamma = {}
    itemGamma = {}
    for u,b,r in ratingsTrain:
        userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]
        itemGamma[b] = [random.random() * 0.1 - 0.05 for k in range(K)]
    def unpack(theta):
        global alpha
        global userBiases
        global itemBiases
        global userGamma
        global itemGamma
        index = 0
        alpha = theta[index]
        index += 1
        userBiases = dict(zip(users, theta[index:index+nUsers]))
        index += nUsers
        itemBiases = dict(zip(items, theta[index:index+nItems]))
        index += nItems
        for u in users:
            userGamma[u] = theta[index:index+K]
            index += K
        for i in items:
            itemGamma[i] = theta[index:index+K]
            index += K
    
    def cost(theta, labels, lamb):
        unpack(theta)
        predictions = [prediction(u, b) for u,b,_ in ratingsTrain]
        cost = MSE(predictions, labels)
        print("MSE = " + str(cost))
        for u in users:
            cost += lamb*userBiases[u]**2
            for k in range(K):
                cost += lamb*userGamma[u][k]**2
        for i in items:
            cost += lamb*itemBiases[i]**2
            for k in range(K):
                cost += lamb*itemGamma[i][k]**2
        return cost
    
    def derivative(theta, labels, lamb):
        unpack(theta)
        N = len(ratingsTrain)
        dalpha = 0
        dUserBiases = defaultdict(float)
        dItemBiases = defaultdict(float)
        dUserGamma = {}
        dItemGamma = {}
        for u in ratingsPerUser:
            dUserGamma[u] = [0.0 for k in range(K)]
        for i in ratingsPerItem:
            dItemGamma[i] = [0.0 for k in range(K)]
        for u,b,r in ratingsTrain:
            pred = prediction(u, i)
            diff = pred - r
            dalpha += 2/N*diff
            dUserBiases[u] += 2/N*diff
            dItemBiases[i] += 2/N*diff
            for k in range(K):
                dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
                dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
        for u in userBiases:
            dUserBiases[u] += 2*lamb*userBiases[u]
            for k in range(K):
                dUserGamma[u][k] += 2*lamb*userGamma[u][k]
        for i in itemBiases:
            dItemBiases[i] += 2*lamb*itemBiases[i]
            for k in range(K):
                dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
        dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
        for u in users:
            dtheta += dUserGamma[u]
        for i in items:
            dtheta += dItemGamma[i]
        return numpy.array(dtheta)
    
    scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(nUsers+nItems) + # Initialize beta
                                   [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
                             derivative, args = (labels, l))
    
for i in range(1,21):
    for j in range(1,21):
        K=i
        l=1e-5*j
        train_model(K,l)
        predictions=[]
        
        for u,b,r in ratingsValid:
            predict=prediction(u, b)
            predictions.append(predict)
        validMSE=MSE(predictions,validlabels)
        if validMSE<bestmse:
            bestmse=validMSE
            bestl=l
            bestK=K
(bestl,bestK)

MSE = 1.741595507037897
MSE = 2.0349250262732665
MSE = 1.764327633475474
MSE = 1.7432837589200663
MSE = 1.7416200771180708
MSE = 1.7414922465040894
MSE = 1.741532731352937
MSE = 1.7414897048023945
MSE = 1.7413176229154375
MSE = 1.740629684411046
MSE = 1.7378841550886046
MSE = 1.7270016329317142
MSE = 1.6850650671249499
MSE = 1.5428152135454951
MSE = 1.2855003052603224
MSE = 1.2482250294852628
MSE = 1.27079463358865
MSE = 1.2498813256389045
MSE = 1.2480925563933933
MSE = 1.2494904930886637
MSE = 1.2498813256389045
MSE = 16.535340637348696
MSE = 1.2489020492654779
MSE = 1.2497288517053773
MSE = 1.2498592105855342
MSE = 1.2498781458928845
MSE = 1.2498808690088514
MSE = 1.2498812600756732
MSE = 1.2498813162255318
MSE = 1.2498813242873654
MSE = 1.2498813254448546
MSE = 1.2498813256110433
MSE = 1.2498813256349042
MSE = 1.2498813256383303
MSE = 1.2498813256388221
MSE = 1.2498813256388928
MSE = 1.2498813256389028
MSE = 1.2498813256389045
MSE = 1.2498813256389045
MSE = 1.2498813256389045
MSE = 

In [144]:
(bestl,bestK)

(1e-05, 5)

In [147]:
train_model(1,1e-4)
predictions=[]

for u,b,r in ratingsValid:
    predict=prediction(u, b)
    predictions.append(predict)
validMSE=MSE(predictions,validlabels)
validMSE

MSE = 1.7416366327907176
MSE = 2.138856369595786
MSE = 1.7728557569854244
MSE = 1.7440743414599977
MSE = 1.7417160487453098
MSE = 1.741517742779776
MSE = 1.7415526992433716
MSE = 1.741520848527638
MSE = 1.7413934589872726
MSE = 1.7408841139868925
MSE = 1.7388501445628501
MSE = 1.7307688361167357
MSE = 1.699316711023215
MSE = 1.4683821885561987
MSE = 1.4638607976259193
MSE = 1.4660994883194032
MSE = 1.4695846502673628
MSE = 1.4653014424481208
MSE = 1.4646693745277821
MSE = 1.4637516590416209
MSE = 1.4642486201415847
MSE = 1.4645817515708262
MSE = 1.4646693745277821
MSE = 1.473941508303604
MSE = 1.4641781422149742
MSE = 1.4645678682231118
MSE = 1.4646480938068374
MSE = 1.4646648951181038
MSE = 1.464668430824039
MSE = 1.4646691756751646
MSE = 1.4646693326248916
MSE = 1.4646693656977907
MSE = 1.4646693726670788
MSE = 1.4646693741356829
MSE = 1.4646693744451562
MSE = 1.4646693745103723
MSE = 1.4646693745241133
MSE = 1.4646693745270076
MSE = 1.4646693745276191
MSE = 1.4646693745274915
MSE = 

1.5210510651360172

In [ ]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    
    predictions.write(u + ',' + b + ',' + str(prediction(u,b)) + '\n')
    
predictions.close()

Try SVD from surprise

SVD1

In [12]:
import surprise
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

In [13]:
f = gzip.GzipFile("train_Interactions.csv.gz")
f

<gzip _io.BufferedReader name='train_Interactions.csv.gz' 0x7f99428f3eb0>

In [14]:
allRatings[0]

('u67805239', 'b61372131', 4)

In [15]:
fname="train_data"


with open(fname, 'w', newline='') as file:
    #file.write('user item rating'+'\n')
    for datum in allRatings:
        case=[str(t) for t in datum]
        
        file.write(' '.join(case)+'\n')


In [16]:
reader = Reader(line_format='user item rating', sep=' ')
data = Dataset.load_from_file("train_data", reader=reader)

In [167]:
trainset, testset = train_test_split(data, test_size=.05)

In [85]:
model1=surprise.SVD(n_factors=1024,n_epochs=2048,lr_all=0.01)
model1.fit(trainset)

In [111]:
#model.fit(trainset)
predictions = model1.test(testset)

In [112]:
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print(sse / len(predictions))

1.4894398806405453


In [113]:
s=model1.predict('u67805239', 'b61372131')
s.est

3.8380020477764303

In [115]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    
    predictions.write(u + ',' + b + ',' + str(model1.predict(u,b).est) + '\n')
    
predictions.close()

SVD exp

In [108]:
model=surprise.SVD(n_factors=2048,n_epochs=10000,lr_all=0.01)
model.fit(trainset)

In [109]:
#model.fit(trainset)
predictions = model.test(testset)

In [110]:
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print(sse / len(predictions))

1.5156262509156995


In [152]:
modele=surprise.SVD(n_factors=1,n_epochs=130,lr_all=0.001,reg_pu=1e-3,reg_qi=1e-3)
modele.fit(trainset)

In [153]:
predictions = modele.test(testset)

In [154]:
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

In [155]:
print(sse / len(predictions))

1.4644212361639146


In [171]:
model=surprise.SVD(n_factors=2,n_epochs=120,lr_all=0.001,reg_pu=1e-4,reg_qi=1e-4)
model.fit(trainset)
predictions = model.test(testset)
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

see=(sse / len(predictions))
see

1.4319951742312254

In [174]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    
    predictions.write(u + ',' + b + ',' + str(model.predict(u,b).est) + '\n')
    
predictions.close()